In [1]:
!pip install eli5

     |████████████████████████████████| 112kB 2.8MB/s 


In [2]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [3]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix'

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [4]:
df=pd.read_csv('data/men_shoes.csv',low_memory=False) #wczytujemy dane
df.shape

(18280, 48)

In [5]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension', 'ean', 'features',
       'flavors', 'imageurls', 'isbn', 'keys', 'manufacturer',
       'manufacturernumber', 'merchants', 'name', 'prices_amountmin',
       'prices_amountmax', 'prices_availability', 'prices_color',
       'prices_condition', 'prices_count', 'prices_currency',
       'prices_dateadded', 'prices_dateseen', 'prices_flavor', 'prices_issale',
       'prices_merchant', 'prices_offer', 'prices_returnpolicy',
       'prices_shipping', 'prices_size', 'prices_source', 'prices_sourceurls',
       'prices_warranty', 'quantities', 'reviews', 'sizes', 'skus',
       'sourceurls', 'upc', 'vin', 'websiteids', 'weight'],
      dtype='object')

In [0]:
def run_model(feats,  model=DecisionTreeRegressor(max_depth=5)): 
  X=df[ feats ].values
  y=df['prices_amountmin'].values
  scores=cross_val_score(model,X,y,scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)
  #model występuje jako parametr zewnętrznt, który mozna zmieniac później

In [7]:
df['brand_cat']=df['brand'].factorize()[0]
run_model(['brand_cat'])

(-58.38655694633361, 4.223555478221712)

In [8]:
model=RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'],model) 

(-57.47223572384038, 4.328288468270897)

In [9]:
df['brand_cat']=df['brand'].map(lambda x: str(x).lower()).factorize()[0]
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [10]:
model=RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'],model)

(-57.31783843165656, 4.181246596160967)

In [11]:
df.features.head().values

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SER

In [12]:
str_dict= '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]'
literal_eval(str_dict)[0]['value'][0]
      

'Men'

In [0]:
def parse_features(x): # definujemy fukncję wyciągającą z features wartości
  output_dict={}
  if str(x)=='nan':return output_dict

  features= literal_eval(x.replace('\\"','"'))
  for item in features:
    key=item['key'].lower().strip()
    value=item['value'][0].lower().strip()
    output_dict[key]=value
    
  return output_dict

df['features_parsed']=df['features'].map(parse_features)

In [15]:
#df['features_parsed'].head().values

array([{'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'}],
      dtype=object)

In [16]:
keys=set()

df['features_parsed'].map(lambda x: keys.update(x.keys()) )

len(keys)

476

In [17]:
def get_name_feat(key): #funkcja nadająca nazwę kolumnom
  return 'feat_'+ key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)]=df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)


In [18]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_metal stamp', 'feat_lens color', 'feat_model number',
       'feat_picture', 'feat_ul safety listing', 'feat_fabrication',
       'feat_fabric material', 'feat_date', 'feat_batteries required',
       'feat_interior pockets'],
      dtype='object', length=526)

In [21]:
#df[False== df['feat_athlete'].isnull()].shape[0]/df.shape[0] * 100 #liczba wszytskich kolumn

0.0437636761487965

In [0]:
keys_stat={}
for key in keys:
 keys_stat[key]= df[False== df[get_name_feat(key)].isnull()].shape[0]/df.shape[0] * 100 #liczba wszytskich kolumn

In [23]:

{k:v for k,v in keys_stat.items() if v > 30} #filt

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat']=df['feat_brand'].factorize()[0]
df['feat_color_cat']=df['feat_color'].factorize()[0]
df['feat_gender_cat']=df['feat_gender'].factorize()[0]
df['feat_manufacturer part number_cat']=df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat']=df['feat_material'].factorize()[0]


df['feat_sport_cat']=df['feat_sport'].factorize()[0] #dodajemy po przetestowaniu nike
df['feat_style_cat']=df['feat_style'].factorize()[0]

for key in keys: #43 minuta optymalizujemy - przelatujemy wszystkie obserwacje jakie mamy
  df[ get_name_feat(key) + '_cat']=df[get_name_feat(key)].factorize()[0]
 

In [73]:
df['brand']=df['brand'].map(lambda x: str(x).lower() ) #normalizacja 
df[df.brand == df.feat_brand].shape

(8846, 1002)

In [0]:
feats=['']


In [74]:
model=RandomForestRegressor(max_depth=5,n_estimators=100)
run_model(['brand_cat'],model)

(-57.38159886194247, 4.1918948378069025)

In [76]:
#feats_cat=[x for x in df.columns if '_cat' in x] #wyciąganie wszytskich parametrów
feats_cat = [x for x in df.columns if '_cat' in x and x != 'feat_catalog']  #feats_cat wpadła nam też kolumna 'feat_catalog', bo ma "cat" w nazwie..
feats_cat

['brand_cat',
 'feat_brand_cat',
 'feat_color_cat',
 'feat_gender_cat',
 'feat_manufacturer part number_cat',
 'feat_material_cat',
 'feat_sport_cat',
 'feat_style_cat',
 'feat_interior pockets_cat',
 'feat_sku#_cat',
 'feat_style #_cat',
 'feat_machine washable_cat',
 'feat_weather-resistant_cat',
 'feat_main colour_cat',
 'feat_front style_cat',
 'feat_display_cat',
 'feat_compatible devices_cat',
 'feat_lens socket width_cat',
 'feat_band width_cat',
 'feat_heart rate monitor_cat',
 'feat_release_cat',
 'feat_contained battery type_cat',
 'feat_width_cat',
 'feat_case type_cat',
 'feat_year_cat',
 'feat_shipping to usa_cat',
 'feat_footwear type_cat',
 'feat_chain/necklace length (in.)_cat',
 'feat_waist size_cat',
 'feat_dial material_cat',
 'feat_feature_cat',
 'feat_cleaning, care & maintenance_cat',
 'feat_series name_cat',
 'feat_power type_cat',
 'feat_eye protection type_cat',
 'feat_colour_cat',
 'feat_sku_cat',
 'feat_year made_cat',
 'feat_hat style_cat',
 'feat_shipping w

In [83]:
df['weight'].unique() # lbs 453.59237  znormalizować tę, i spr czy ma wpływ czy nie 

array([nan, '3.0 lbs', '9 g', '1.45 lbs', '0.45 lbs', '1.0 lbs',
       '0.23 lbs', '5.0 lbs', '5.5 lbs', '7.45 lbs', '4.0 lbs',
       '2.7969 lbs', '3.9 lbs', '4.6 pounds', '2.1 lbs', '1.1057 lbs',
       '15.0 lbs', '2.4 ounces', '454 g', '0.105 lbs', '9.1 ounces',
       '4.8 lbs', '6.1 lbs', '6.5 lbs', '1.1041 lbs', '1.3 Kg', '91 g',
       '20.0 lbs', '6.0 lbs', '386 g', '0.81 lbs', '4.5 lbs',
       '0.5 ounces', '2.0 lbs', '3.13 lbs', '5.9 lbs', '6.15 lbs',
       '1 pounds', '1.95 lbs', '2.15 lbs', '2 pounds', '2.1 pounds',
       '14 Kg', '0.4788 lbs', '10.0 lbs', '0.38 lbs', '2.5 lbs',
       '68.912 lbs', '45 g', '13.09 lbs', '2.5 pounds', '0.21 lbs',
       '16.75 lbs', '6.3 lbs', '272 g', '1.8 Kg', '2.8 pounds', '0.1 lbs',
       '5.05 lbs', '0.28 lbs', '76.08 lbs', '0.15 lbs', '200 g',
       '7.8 pounds', '399 g', '4.95 lbs', '64.144 lbs', '24 pounds',
       '73.696 lbs', '1.6 lbs', '6.6 ounces', '5 g', '1.2 Kg', '862 g',
       '3.05 lb', '8.6 ounces', '3.6 lbs', '71.

In [0]:
#feats=['brand_cat','feat_brand_cat','feat_gender_cat','feat_material_cat','feat_style_cat','feat_sport_cat'] #usunęlismy 'feat_manufacturer part number_cat','feat_color_cat'
feats=['brand_cat','feat_metal type_cat','feat_shape_cat','feat_brand_cat','feat_gender_cat','feat_material_cat','feat_style_cat','feat_sport_cat'] #usunęlismy 'feat_manufacturer part number_cat','feat_color_cat'
#feats += feats_cat
#feats=list(set(feats))

model=RandomForestRegressor(max_depth=5,n_estimators=100)
result=run_model(feats,model)

In [85]:
X=df[feats].values
y=df['prices_amountmin'].values
m=RandomForestRegressor(max_depth=5,n_estimators=100, random_state=0)
m.fit(X,y)

print(result)
perm=PermutationImportance(m, random_state=1).fit(X,y);
eli5.show_weights(perm, feature_names=feats)


(-57.29997241192092, 4.254617041154466)


Weight,Feature
0.2557 ± 0.0080,brand_cat
0.1032 ± 0.0075,feat_material_cat
0.0252 ± 0.0026,feat_gender_cat
0.0175 ± 0.0015,feat_brand_cat
0.0127 ± 0.0011,feat_shape_cat
0.0091 ± 0.0015,feat_metal type_cat
0.0033 ± 0.0012,feat_style_cat
0.0002 ± 0.0000,feat_sport_cat


In [49]:
#df['brand'].value_counts(normalize=True) #poznajemy lepiej brand
#df[df['brand']=='nike'].features_parsed.head().values #patrzymy na dane z nike
df[df['brand']=='nike'].features_parsed.sample(5).values #patrzymy na dane z nike

#df['feat_age_group].value_counts()

array([{'sport': 'basketball', 'gender': 'boys', 'size': '6', 'shoe category': 'boys shoes', 'color': 'dark grey/black/wolf grey/white', 'casual & dress shoe style': 'athletic', 'brand': 'nike', 'manufacturer_part_number': '807549-003', 'shoe width': 'medium'},
       {'style': 'basketball shoes', 'pattern': 'solid', 'material': 'synthetic', 'condition': 'new with box'},
       {'sport': 'soccer', 'type': 'cleats', 'color': 'fluo yellow/black', 'condition': 'new with box', 'ean': '0886059006695'},
       {'sport': 'fitness & yoga, swimming, tennis, volleyball, yoga, baseball & softball, basketball, bowling, camping & hiking, cycling, equestrian, fishing, fitness, football, golf, gymnastics, hockey, martial arts, paintball, running, skateboarding, ski & snow, soccer, surfing & windsurfing', 'style': 'vests', 'condition': 'new with tags', 'sleeve length': 'sleeveless'},
       {'condition': 'new with box', 'type': 'cleats'}], dtype=object)

In [87]:
ls matrix_one

day3.ipynb  day4.ipynb  day5.ipynb
